可以在培训期间和培训后保存模型进度。这意味着模型可以从中断的地方恢复，并避免长时间的训练。保存也意味着您可以共享您的模型，而其他人可以重新创建您的工作。在发布研究模型和技术时，大多数机器学习从业者分享：
- 用于创建模型的代码，以及
- 模型的训练重量或参数

共享此数据有助于其他人了解模型的工作原理，并使用新数据自行尝试。

## 建立

安装和导入

安装并导入TensorFlow和依赖项：

In [2]:
!pip install -q h5py pyyaml 

In [4]:
from __future__ import absolute_import ,print_function, division
import os

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

1.10.0


## 获取示例数据集
我们将使用MNIST数据集来训练我们的模型来演示保存权重。要加速这些演示运行，请仅使用前1000个示例：

In [5]:
(train_images,train_labels),(test_images,test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

## 定义模型

In [11]:
def create_model():
    model = keras.models.Sequential([
        keras.layers.Dense(512,activation=tf.nn.relu,input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10,activation=tf.nn.softmax)
    ])
    
    model.compile(optimizer = tf.keras.optimizers.Adam(),
                 loss = keras.losses.sparse_categorical_crossentropy,
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [12]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = keras.callbacks.ModelCheckpoint(checkpoint_path,
                                              save_weights_only=True,
                                              verbose=1)
model = create_model()

model.fit(train_images,train_labels,epochs=10,validation_data=(test_images,test_labels),
          callbacks=[cp_callback])# pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 1s 970us/step - loss: 1.1784 - acc: 0.6540 - val_loss: 0.7015 - val_acc: 0.7980

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
1000/1000 [==============================] - 0s 494us/step - loss: 0.4357 - acc: 0.8790 - val_loss: 0.5193 - val_acc: 0.8380

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
1000/1000 [==============================] - 0s 458us/step - loss: 0.2921 - acc: 0.9230 - val_loss: 0.4667 - val_acc: 0.8620

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
1000/1000 [==============================] - 0s 464us/step - loss: 0.2149 - acc: 0.9500 - val_loss: 0.4207 - val_acc: 0.8630

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
1000/1000 [==============================] - 0s 466us/step - loss: 0.1541 - acc: 0.9650 - val_loss: 0.4267 - val_acc: 0.8600

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
1000/1000 [=====

这将创建一个TensorFlow检查点文件集合，这些文件在每个时期结束时更新：

现在重建一个新的未经训练的模型，并在测试集上进行评估。未经训练的模型将在偶然水平上执行（准确度约为10％）：

In [16]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels)
print('Untrain model,accuracy:{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 133us/step
Untrain model,accuracy:10.40%


然后从检查点加载权重，并重新评估：

In [17]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels)
print('Untrain model,accuracy:{:5.2f}%'.format(100*acc))

1000/1000 [==============================] - 0s 46us/step
Untrain model,accuracy:87.00%


## 保存整个模型

整个模型可以保存到包含权重值，模型配置甚至优化器配置的文件中。这允许您检查模型并稍后从完全相同的状态恢复培训 - 无需访问原始代码。

在Keras中保存功能齐全的模型非常有用 - 您可以在TensorFlow.js中加载它们，然后在Web浏览器中训练和运行它们。

Keras使用HDF5标准提供基本保存格式。出于我们的目的，可以将保存的模型视为单个二进制blob。

In [18]:
model = create_model()

model.fit(train_images,train_labels,epochs=5)

# Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 1s 597us/step - loss: 1.1872 - acc: 0.6650
Epoch 2/5
1000/1000 [==============================] - 0s 360us/step - loss: 0.4401 - acc: 0.8660
Epoch 3/5
1000/1000 [==============================] - 0s 363us/step - loss: 0.2877 - acc: 0.9270
Epoch 4/5
1000/1000 [==============================] - 0s 440us/step - loss: 0.2154 - acc: 0.9440
Epoch 5/5
1000/1000 [==============================] - 0s 344us/step - loss: 0.1491 - acc: 0.9650


现在从该文件重新创建模型：

In [19]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
loss,acc = new_model.evaluate(test_images,test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 163us/step
Restored model, accuracy: 86.80%


这项技术可以节省一切

- 重量值
- 模型的配置（架构）
- 优化器配置

Keras通过检查架构来保存模型。目前，它无法保存TensorFlow优化器（来自tf.train）。使用这些时，您需要在加载后重新编译模型，并且您将失去优化器的状态。